# Scorigami Analysis

A Scorigami is a final score that has never occurred in the history of the NFL. We will make a sp data frame to determine the number of scoring plays in each game of the NFL season.

In [1]:
import pandas as pd
import numpy as np
import altair as alt

We start by loading the dataset of all plays in the 2023 NFL season. We extract the columns that we want and we create a all_games dataframe of all scoring plays including failed extra points and two point conversions. From there, we assign 6 for all touchdowns and 1 for sucessful extra points and 2 for sucessful 2 point conversions. 

In [2]:
all_plays_2023 = pd.read_csv('/Users/MC/Downloads/play_by_play_2023.csv')
plays_2023 = all_plays_2023[['season_type','game_id','week','qtr','total_away_score','total_home_score','quarter_seconds_remaining','sp','posteam','defteam','side_of_field','home_team','away_team','td_team','touchdown','extra_point_result','two_point_conv_result','field_goal_result','total_home_score','total_away_score','desc','play_type','safety']].copy()
all_games = plays_2023[(plays_2023['sp'] == 1) | (plays_2023['extra_point_result'] == 'failed') | (plays_2023['two_point_conv_result'] == 'failure')].copy()
all_games['home_team_score'] = (all_games['td_team'] == all_games['home_team']).map(lambda x:  6 if x == True else 0)
all_games['away_team_score'] = (all_games['td_team'] == all_games['away_team']).map(lambda x:  6 if x == True else 0)
all_games['xp'] = all_games['extra_point_result'].map(lambda x:  1 if x == 'good' else 0).shift(-1).fillna(0)
all_games['2pt'] = all_games['two_point_conv_result'].map(lambda x:  2 if x == 'success' else 0).shift(-1).fillna(0)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/MC/Downloads/play_by_play_2023.csv'

Next, we create a pandas series for all safetys (assigned 2 points) and field goal kicks (assigned 3 points).

In [3]:
homesafety = ((all_games['defteam'] == all_games['home_team']) & (all_games['safety'] == 1)).map(lambda x: 2 if x == True else 0)
awaysafety = ((all_games['defteam'] == all_games['away_team']) & (all_games['safety'] == 1)).map(lambda x: 2 if x == True else 0)
homekick = ((all_games['posteam'] == all_games['home_team']) & (all_games['field_goal_result'] == 'made')).map(lambda x:  3 if x == True else 0)
awaykick = ((all_games['posteam'] == all_games['away_team']) & (all_games['field_goal_result'] == 'made')).map(lambda x:  3 if x == True else 0)

We make a column in all_games for the total home scoring pocession. For example, if a team scored 6 points and a 1 point extra point, we record that play as 7 points.

In [4]:
two_h = ((homesafety == 2)).map(lambda x:  2 if x == True else 0)
three_h = ((homekick == 3)).map(lambda x:  3 if x == True else 0)
six_h = ((all_games['home_team_score'] == 6) & (all_games['xp'] == 0) & (all_games['2pt'] == 0)).map(lambda x:  6 if x == True else 0)
seven_h = ((all_games['home_team_score'] == 6) & (all_games['xp'] == 1)).map(lambda x:  7 if x == True else 0)
eight_h = ((all_games['home_team_score'] == 6) & (all_games['2pt'] == 2)).map(lambda x:  8 if x == True else 0)
all_games['home_score'] = two_h + three_h + six_h + seven_h + eight_h

We do the same for the away team score. 

In [5]:
two_a = ((awaysafety == 2)).map(lambda x:  2 if x == True else 0)
three_a = ((awaykick == 3)).map(lambda x:  3 if x == True else 0)
six_a = ((all_games['away_team_score'] == 6) & (all_games['xp'] == 0) & (all_games['2pt'] == 0)).map(lambda x:  6 if x == True else 0)
seven_a = ((all_games['away_team_score'] == 6) & (all_games['xp'] == 1)).map(lambda x:  7 if x == True else 0)
eight_a = ((all_games['away_team_score'] == 6) & (all_games['2pt'] == 2)).map(lambda x:  8 if x == True else 0)
all_games['away_score'] = two_a +three_a + six_a + seven_a + eight_a
all_games

,season_type,game_id,week,qtr,total_away_score,total_home_score,quarter_seconds_remaining,sp,posteam,defteam,...,total_away_score,desc,play_type,safety,home_team_score,away_team_score,xp,2pt,home_score,away_score
25,REG,2023_01_ARI_WAS,1,1,0,6,262.0,1,WAS,ARI,...,0,(4:22) (Shotgun) 14-S.Howell pass short left t...,pass,0.0,6,0,1.0,0.0,7,0
26,REG,2023_01_ARI_WAS,1,1,0,7,255.0,1,WAS,ARI,...,0,"6-J.Slye extra point is GOOD, Center-54-C.Chee...",extra_point,0.0,0,0,0.0,0.0,0,0
36,REG,2023_01_ARI_WAS,1,1,3,7,56.0,1,ARI,WAS,...,3,"(:56) 5-M.Prater 28 yard field goal is GOOD, C...",field_goal,0.0,0,0,0.0,0.0,0,3
46,REG,2023_01_ARI_WAS,1,2,6,7,811.0,1,ARI,WAS,...,6,"(13:31) 5-M.Prater 54 yard field goal is GOOD,...",field_goal,0.0,0,0,0.0,0.0,0,3
79,REG,2023_01_ARI_WAS,1,2,12,7,62.0,1,WAS,ARI,...,12,(1:02) (Shotgun) 14-S.Howell sacked at WAS 12 ...,pass,0.0,0,6,1.0,0.0,0,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48370,POST,2023_19_PIT_BUF,19,3,10,24,96.0,1,BUF,PIT,...,10,"(1:36) 2-T.Bass 45 yard field goal is GOOD, Ce...",field_goal,0.0,0,0,0.0,0.0,3,0
48384,POST,2023_19_PIT_BUF,19,4,16,24,637.0,1,PIT,BUF,...,16,(10:37) (Shotgun) 2-M.Rudolph pass short left ...,pass,0.0,0,6,1.0,0.0,0,7
48385,POST,2023_19_PIT_BUF,19,4,17,24,632.0,1,PIT,BUF,...,17,"9-C.Boswell extra point is GOOD, Center-46-C.K...",extra_point,0.0,0,0,0.0,0.0,0,0
48394,POST,2023_19_PIT_BUF,19,4,17,30,397.0,1,BUF,PIT,...,17,(6:37) (Shotgun) 17-J.Allen pass short middle ...,pass,0.0,6,0,1.0,0.0,7,0


Next, we make a sp dataframe copied form the all_games dataframe. For all postive scoring plays, we assign 1 and 0 for non-scoring plays. 

In [6]:
sp = all_games.copy()
sp['home_score'] = sp['home_score'].map(lambda x: 1 if x>0 else 0)
sp['away_score'] = sp['away_score'].map(lambda x: 1 if x>0 else 0)
sp = sp.groupby('game_id',as_index = False).sum()
sp

,game_id,season_type,week,qtr,total_away_score,total_home_score,quarter_seconds_remaining,sp,posteam,defteam,...,total_away_score,desc,play_type,safety,home_team_score,away_team_score,xp,2pt,home_score,away_score
0,2023_01_ARI_WAS,REGREGREGREGREGREGREGREGREGREGREG,11,26,111,114,3684.0,11,WASWASARIARIWASARIWASARIWASWASWAS,ARIARIWASWASARIWASARIWASARIARIARI,...,111,(4:22) (Shotgun) 14-S.Howell pass short left t...,passextra_pointfield_goalfield_goalpassextra_p...,0.0,12,6,3.0,0.0,4,4
1,2023_01_BUF_NYJ,REGREGREGREGREGREGREGREGREGREGREG,11,33,122,97,3236.0,11,BUFNYJBUFBUFBUFNYJNYJNYJNYJBUFBUF,NYJBUFNYJNYJNYJBUFBUFBUFBUFNYJNYJ,...,122,"(3:09) 2-T.Bass 40 yard field goal is GOOD, Ce...",field_goalfield_goalpassextra_pointfield_goalf...,0.0,12,6,2.0,0.0,5,4
2,2023_01_CAR_ATL,REGREGREGREGREGREGREGREGREGREG,10,30,73,124,5652.0,10,ATLATLCARCARCARATLATLATLATLATL,CARCARATLATLATLCARCARCARCARCAR,...,73,(15:00) (Shotgun) 9-D.Ridder pass short right ...,passextra_pointpassextra_pointfield_goalfield_...,0.0,18,6,4.0,0.0,4,2
3,2023_01_CIN_CLE,REGREGREGREGREGREGREGREG,8,24,15,107,3549.0,8,CLECLECLECINCLECLECLECLE,CINCINCINCLECINCINCINCIN,...,15,(14:19) 7-D.Hopkins 42 yard field goal is GOOD...,field_goalrunextra_pointfield_goalfield_goalfi...,0.0,12,0,1.0,2.0,5,1
4,2023_01_DAL_NYG,REGREGREGREGREGREGREGREGREGREGREGREG,12,25,266,0,5697.0,11,NYGDALDALNYGDALDALDALDALDALDALDALDAL,DALNYGNYGDALNYGNYGNYGNYGNYGNYGNYGNYG,...,266,(8:14) 9-G.Gano 45 yard field goal is BLOCKED ...,field_goalextra_pointfield_goalpassextra_point...,0.0,0,30,4.0,0.0,0,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273,2023_19_GB_DAL,POSTPOSTPOSTPOSTPOSTPOSTPOSTPOSTPOSTPOSTPOSTPO...,437,63,717,247,7514.0,21,GBGBGBGBGBGBDALGBDALDALDALGBGBDALDALGBGBGBGBDA...,DALDALDALDALDALDALGBDALGBGBGBDALDALGBGBDALDALD...,...,717,(7:12) (Shotgun) 33-A.Jones right guard for 3 ...,runextra_pointrunextra_pointpassextra_pointpas...,0.0,24,42,7.0,4.0,5,7
274,2023_19_LA_DET,POSTPOSTPOSTPOSTPOSTPOSTPOSTPOSTPOSTPOSTPOSTPO...,266,27,141,230,5749.0,14,DETDETLADETDETLALADETDETLALADETLALA,LALADETLALADETDETLALADETDETLADETDET,...,141,(9:33) 5-D.Montgomery up the middle for 1 yard...,runextra_pointfield_goalrunextra_pointpassextr...,0.0,18,12,5.0,0.0,4,5
275,2023_19_MIA_KC,POSTPOSTPOSTPOSTPOSTPOSTPOSTPOSTPOSTPOST,190,23,48,142,6213.0,10,KCKCKCMIAMIAKCKCKCKCKC,MIAMIAMIAKCKCMIAMIAMIAMIAMIA,...,48,(11:10) (Shotgun) 15-P.Mahomes pass short righ...,passextra_pointfield_goalpassextra_pointfield_...,0.0,12,6,3.0,0.0,6,1
276,2023_19_PHI_TB,POSTPOSTPOSTPOSTPOSTPOSTPOSTPOSTPOSTPOSTPOSTPO...,247,30,69,226,4577.0,12,TBTBTBTBPHITBPHIPHIPHITBTBTBTB,PHIPHIPHIPHITBPHITBTBTBPHIPHIPHIPHI,...,69,(10:05) 4-C.McLaughlin 28 yard field goal is G...,field_goalpassextra_pointfield_goalfield_goalf...,1.0,18,6,3.0,0.0,7,2


Below is a dictionary of the likelihood of how many scoring plays a team scores in a given NFL game.

In [7]:
num_scores = {place:sp[place].value_counts() for place in ['home_score','away_score']}
num_scores['home_score'] = num_scores['home_score'] / sum(num_scores['home_score'] )
num_scores['away_score'] = num_scores['away_score'] / sum(num_scores['away_score'] )
num_scores

{'home_score': home_score
 4     0.212230
 5     0.212230
 3     0.172662
 6     0.154676
 2     0.097122
 7     0.075540
 1     0.028777
 8     0.021583
 0     0.017986
 10    0.003597
 9     0.003597
 Name: count, dtype: float64,
 'away_score': away_score
 4    0.241007
 3    0.223022
 5    0.147482
 2    0.129496
 6    0.122302
 1    0.064748
 7    0.050360
 8    0.010791
 0    0.010791
 Name: count, dtype: float64}

Next, we will create a dictionary of the probability that a single scoring play results in 2,3,6,7, or 8 points.

In [8]:
multiser = pd.read_html('/Users/MC/Downloads/scoregami.rtf',skiprows = 2)[0].fillna(0).stack()
multiser = pd.DataFrame(multiser)
scorigami = multiser[multiser[0] == 0]
scorigami

0
0  1   0.0
   4   0.0
   46  0.0
   50  0.0
   61  0.0
...    ...
73 69  0.0
   70  0.0
   71  0.0
   72  0.0
   73  0.0

[4393 rows x 1 columns]

For each game, we will check if it is a scorigami. If it is, we will add it to the scori list. We will also count how many games it generates 1000 scorigamis.

In [9]:
len_h = (all_games['home_score'] > 0).map(lambda x: 1 if x == True else 0).sum()
len_a = (all_games['away_score'] > 0).map(lambda x: 1 if x == True else 0).sum()
scores = {'home_score': [len_h], 'away_score': [len_a]}

In [10]:
plays = {place:{i:(all_games[[place]] == i).sum()[place]/scores[place][0] for score in [[len_h,len_a]] for i in [2,3,6,7,8]} for place in ['home_score','away_score']}
plays

{'home_score': {2: 0.010752688172043012,
  3: 0.39619520264681557,
  6: 0.04218362282878412,
  7: 0.5153019023986766,
  8: 0.03556658395368073},
 'away_score': {2: 0.004651162790697674,
  3: 0.41674418604651164,
  6: 0.0586046511627907,
  7: 0.4930232558139535,
  8: 0.026976744186046512}}

We create a games function that returns a list of scores for the number of games and team inputted.

In [11]:
rng = np.random.default_rng()
def games(n,team):
    i = 1
    if (team == 'home'): i = 0
    scores_in_game = rng.choice(a=list(plays[f"{team}_score"].keys()), size=n, p=pd.DataFrame(plays.values()).iloc[i])
    return scores_in_game

The following num_sp function returns how many scoring plays occured for a given team for a number of games inputted.

In [12]:
rng = np.random.default_rng()
def num_sp(n,team):
    sp = rng.choice(a=list(num_scores[f"{team}_score"].index), size=n, p=list(num_scores[f"{team}_score"].values))
    return sp

The scores function uses the previous two functions to generate final scores for both the home and away team.

In [13]:
def scores(n):
    home_score_plays = num_sp(n,'home')
    away_score_plays = num_sp(n,'away')
    scores_list = []
    for i in range(len(home_score_plays)):
        h = int(sum(games(home_score_plays[i],'home')))
        a = int(sum(games(away_score_plays[i],'away')))
        if h < a:
            scores_list.append((h,a))
        else:
            scores_list.append((a,h))
    return scores_list

The following code simulates scores and adds them to the scori dataframe if that particular score is a scorigami. We count how many games it takes to get 1000 scorigamis. We see that it will take an estimated 38,000 games to produce 1000 scorigamis. We predict we see a new scorigami every 38 to 40 games.

In [14]:
scori = []
temp_array = []
counter = 0
while len(scori) < 1000:
    counter += 1
    temp = scores(1)[0]
    temp_array.append(temp)
    if temp in scorigami.index:
        scori.append(temp)
scori = pd.DataFrame({"score":scori})
counter

38755

Finally, we will generate a histogram of scorigamis to determine which ones are the most likely to occur next. We predict that the next scorgiami is 23-26 with 30/1000 of the simulated scorigamis. 26-32 and 16-39 are also likley next scorigmis.

In [15]:
hist = alt.Chart(scori).mark_bar().encode(
    x = alt.X('score:N',
                sort=alt.EncodingSortField(field="Letters", op="count", order='descending'),
                axis=alt.Axis(labelAngle=0)), 
    y = 'count()',
    tooltip=['score', 'count()']).interactive()
hist

/Users/mc/Downloads/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)